In [ ]:
import cv2
import numpy as np


def getYellow(color):
    blurred = cv2.GaussianBlur(color, (3, 3), 0)

    ## Convert to HSV
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, (15, 25, 25), (50, 255,255))

    ## Slice the Blue
    imask = mask>0
    blue = np.zeros_like(blurred, np.uint8)
    blue[imask] = blurred[imask]

    gray = cv2.cvtColor(blue, cv2.COLOR_BGR2GRAY) 

    (T, thresh) = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY) 

    # Change to look at regions not ignore regions
    
    thresh[:40,:]=0 #top
    thresh[70:,:60]=0 #bottom left
    thresh[70:,100:]=0 #bottom right
    
    return thresh

def getBlue(color):
    blurred = cv2.GaussianBlur(color, (3, 3), 0)

    ## Convert to HSV
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    
    mask = cv2.inRange(hsv, (90, 25, 25), (130, 255,255))

    ## Slice the Blue
    imask = mask>0
    blue = np.zeros_like(blurred, np.uint8)
    blue[imask] = blurred[imask]

    gray = cv2.cvtColor(blue, cv2.COLOR_BGR2GRAY)

    (T, thresh) = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)
    
    # Change to look at regions not ignore regions
    
    thresh[:40,:]=0 #top
    thresh[70:,:60]=0 #bottom left
    thresh[70:,100:]=0 #bottom right
    
    return thresh

def getWhite(image):
    return np.sum(image == 255)



In [ ]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv 
import cv2
import numpy as np

#using realsense to capture the color image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of this value will be captured by traitlets
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()
        self.flag = 0
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        
    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            self.color_value = image #assign the numpy array image to the color_value variable 

           
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data


In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
import sys
import time
from RobotClass import Robot

#initialize the Robot class
robot = Robot()


#create widgets for the displaying of the image
display_image = widgets.Image(format='jpeg', width='45%') #determine the width of the YELLOW image
#display_blue = widgets.Image(format='jpeg', width='45%')  #determine the width of the BLUE image
layout=widgets.Layout(width='100%')

#sidebyside = widgets.HBox([display_yellow, display_blue],layout=layout) #horizontal 
display(display_image) #display the widget



#callback function, invoked when traitlets detects the changing of the color image
def process(change):
    image = change['new'] #retrieve data from the input dict
    
    line_color = 0
    if getWhite(getYellow(image)) > getWhite(getBlue(image)):
        line_color = 0
    else: line_color = 1
    
    if line_color == 0:
        display_image.value = bgr8_to_jpeg(getYellow(cv2.resize(image,(160,120))))
        output_image = getYellow(cv2.resize(image,(160,120)))
    if line_color == 1:
        display_image.value = bgr8_to_jpeg(getBlue(cv2.resize(image,(160,120))))
        output_image = getBlue(cv2.resize(image,(160,120)))
    
    if getWhite(output_image)>80:
        camera.flag = 1
    if getWhite(output_image)<80:
        camera.flag = 0
    

#processing({'new': camera.color_value})
#the camera.observe function will monitor the color_value variable. If this value changes, the processing function will be excuted.
camera.observe(process, names='color_value')

In [ ]:
#stop observe the changing of the variables
camera.unobserve_all()
camera.stop() #stop data capturing thread
#you will find that the image stops changing in the above window

In [ ]:
#start data capturing thread again
camera.start()
camera.observe(process, names='color_value') #start monitor the changing of the color image 
#you will find that the image start changing again in the above window

In [ ]:
import time
from RobotClass import Robot

#initialize the Robot class
robot = Robot()

#this for loop will probably runs for 10 seconds
for i in range(10000):
    if camera.flag == 1:
        print("go")
        robot.forward(0.3) 
    else:
        print("stop")
        robot.stop() 
        
#stop the robot
robot.stop()


In [ ]:
from RobotClass import Robot
robot = Robot()
robot.stop()